In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [2]:
import pandas as pd

In [3]:
train_data = pd.read_csv("./drive/My Drive/Colab Notebooks/used_car_train_20200313.csv", sep = ' ')
test_data = pd.read_csv("./drive/My Drive/Colab Notebooks/used_car_testB_20200421.csv", sep = ' ')

In [4]:
train_data.describe()

,SaleID,name,regDate,model,brand,bodyType,fuelType,gearbox,power,kilometer,regionCode,seller,offerType,creatDate,price,v_0,v_1,v_2,v_3,v_4,v_5,v_6,v_7,v_8,v_9,v_10,v_11,v_12,v_13,v_14
count,150000.000000,150000.000000,1.500000e+05,149999.000000,150000.000000,145494.000000,141320.000000,144019.000000,150000.000000,150000.000000,150000.000000,150000.000000,150000.0,1.500000e+05,150000.000000,150000.000000,150000.000000,150000.000000,150000.000000,150000.000000,150000.000000,150000.000000,150000.000000,150000.000000,150000.000000,150000.000000,150000.000000,150000.000000,150000.000000,150000.000000
mean,74999.500000,68349.172873,2.003417e+07,47.129021,8.052733,1.792369,0.375842,0.224943,119.316547,12.597160,2583.077267,0.000007,0.0,2.016033e+07,5923.327333,44.406268,-0.044809,0.080765,0.078833,0.017875,0.248204,0.044923,0.124692,0.058144,0.061996,-0.001000,0.009035,0.004813,0.000313,-0.000688
std,43301.414527,61103.875095,5.364988e+04,49.536040,7.864956,1.760640,0.548677,0.417546,177.168419,3.919576,1885.363218,0.002582,0.0,1.067328e+02,7501.998477,2.457548,3.641893,2.929618,2.026514,1.193661,0.045804,0.051743,0.201410,0.029186,0.035692,3.772386,3.286071,2.517478,1.288988,1.038685
min,0.000000,0.000000,1.991000e+07,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.500000,0.000000,0.000000,0.0,2.015062e+07,11.000000,30.451976,-4.295589,-4.470671,-7.275037,-4.364565,0.000000,0.000000,0.000000,0.000000,0.000000,-9.168192,-5.558207,-9.639552,-4.153899,-6.546556
25%,37499.750000,11156.000000,1.999091e+07,10.000000,1.000000,0.000000,0.000000,0.000000,75.000000,12.500000,1018.000000,0.000000,0.0,2.016031e+07,1300.000000,43.135799,-3.192349,-0.970671,-1.462580,-0.921191,0.243615,0.000038,0.062474,0.035334,0.033930,-3.722303,-1.951543,-1.871846,-1.057789,-0.437034
50%,74999.500000,51638.000000,2.003091e+07,30.000000,6.000000,1.000000,0.000000,0.000000,110.000000,15.000000,2196.000000,0.000000,0.0,2.016032e+07,3250.000000,44.610266,-3.052671,-0.382947,0.099722,-0.075910,0.257798,0.000812,0.095866,0.057014,0.058484,1.624076,-0.358053,-0.130753,-0.036245,0.141246
75%,112499.250000,118841.250000,2.007111e+07,66.000000,13.000000,3.000000,1.000000,0.000000,150.000000,15.000000,3843.000000,0.000000,0.0,2.016033e+07,7700.000000,46.004721,4.000670,0.241335,1.565838,0.868758,0.265297,0.102009,0.125243,0.079382,0.087491,2.844357,1.255022,1.776933,0.942813,0.680378
max,149999.000000,196812.000000,2.015121e+07,247.000000,39.000000,7.000000,6.000000,1.000000,19312.000000,15.000000,8120.000000,1.000000,0.0,2.016041e+07,99999.000000,52.304178,7.320308,19.035496,9.854702,6.829352,0.291838,0.151420,1.404936,0.160791,0.222787,12.357011,18.819042,13.847792,11.147669,8.658418


In [5]:
test_data.describe()

,SaleID,name,regDate,model,brand,bodyType,fuelType,gearbox,power,kilometer,regionCode,seller,offerType,creatDate,v_0,v_1,v_2,v_3,v_4,v_5,v_6,v_7,v_8,v_9,v_10,v_11,v_12,v_13,v_14
count,50000.000000,50000.000000,5.000000e+04,50000.00000,50000.000000,48496.000000,47076.000000,48032.000000,50000.000000,50000.000000,50000.000000,50000.0,50000.0,5.000000e+04,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000
mean,224999.500000,68505.606100,2.003401e+07,47.64948,8.087140,1.793736,0.376498,0.226953,119.766960,12.598260,2581.080680,0.0,0.0,2.016033e+07,44.400023,-0.065525,0.079706,0.078381,0.022361,0.248147,0.044624,0.124693,0.058198,0.062113,0.019633,0.002759,0.004342,0.004570,-0.007209
std,14433.901067,61032.124271,5.351615e+04,49.90741,7.899648,1.764970,0.549281,0.418866,206.313348,3.912519,1889.248559,0.0,0.0,1.113395e+02,2.459920,3.636631,2.930829,2.019136,1.194215,0.045836,0.051664,0.201440,0.029171,0.035723,3.764095,3.289523,2.515912,1.287194,1.044718
min,200000.000000,1.000000,1.991000e+07,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.500000,0.000000,0.0,0.0,2.014031e+07,31.122325,-4.231855,-4.032142,-5.801254,-4.233626,0.000000,0.000000,0.000000,0.000000,0.000000,-9.119719,-5.662163,-8.291868,-4.157649,-6.098192
25%,212499.750000,11315.000000,1.999100e+07,11.00000,1.000000,0.000000,0.000000,0.000000,75.000000,12.500000,1006.000000,0.0,0.0,2.016031e+07,43.120935,-3.193169,-0.967832,-1.456793,-0.922153,0.243436,0.000035,0.062519,0.035413,0.033880,-3.675196,-1.963928,-1.865406,-1.048722,-0.440706
50%,224999.500000,52215.000000,2.003091e+07,30.00000,6.000000,1.000000,0.000000,0.000000,110.000000,15.000000,2204.500000,0.0,0.0,2.016032e+07,44.601493,-3.053506,-0.384910,0.118448,-0.068187,0.257818,0.000801,0.095880,0.056804,0.058749,1.632134,-0.375537,-0.138943,-0.036352,0.136849
75%,237499.250000,118710.750000,2.007110e+07,66.00000,13.000000,3.000000,1.000000,0.000000,150.000000,15.000000,3842.000000,0.0,0.0,2.016033e+07,45.987018,3.978703,0.239689,1.563490,0.871565,0.265263,0.101654,0.125470,0.079387,0.087624,2.846205,1.263451,1.775632,0.945239,0.685555
max,249999.000000,196808.000000,2.015121e+07,246.00000,39.000000,7.000000,6.000000,1.000000,19211.000000,15.000000,8120.000000,0.0,0.0,2.016041e+07,51.676686,7.190759,18.865988,9.386558,4.959106,0.291176,0.153403,1.411559,0.157458,0.211304,12.177864,18.789496,13.384828,5.635374,2.649768


In [6]:
train_data["notRepairedDamage"].value_counts()

0.0    111361
-       24324
1.0     14315
Name: notRepairedDamage, dtype: int64

In [7]:
train_data["notRepairedDamage"].replace("-", "0.0", inplace=True)
train_data["notRepairedDamage"] = train_data["notRepairedDamage"].astype("float64")

In [8]:
test_data["notRepairedDamage"].value_counts()

0.0    37224
-       8069
1.0     4707
Name: notRepairedDamage, dtype: int64

In [9]:
test_data["notRepairedDamage"].replace("-", "0.0", inplace=True)
test_data["notRepairedDamage"] = test_data["notRepairedDamage"].astype("float64")

In [10]:
train_data["power"][train_data["power"] > 600] = 600
test_data["power"][test_data["power"] > 600] = 600

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [11]:
# check null value
print(train_data.isnull().sum())
print(test_data.isnull().sum())

SaleID                  0
name                    0
regDate                 0
model                   1
brand                   0
bodyType             4506
fuelType             8680
gearbox              5981
power                   0
kilometer               0
notRepairedDamage       0
regionCode              0
seller                  0
offerType               0
creatDate               0
price                   0
v_0                     0
v_1                     0
v_2                     0
v_3                     0
v_4                     0
v_5                     0
v_6                     0
v_7                     0
v_8                     0
v_9                     0
v_10                    0
v_11                    0
v_12                    0
v_13                    0
v_14                    0
dtype: int64
SaleID                  0
name                    0
regDate                 0
model                   0
brand                   0
bodyType             1504
fuelType             2924

In [12]:
# fill null value
train_data = train_data.fillna(train_data.median())
test_data = test_data.fillna(test_data.median())

In [13]:
# min_max scaler process
from sklearn.preprocessing import MinMaxScaler
min_max_scaler = MinMaxScaler()

features = train_data.columns.to_list()
features.remove('SaleID')
features.remove('price')

In [14]:

train_x = min_max_scaler.fit_transform(train_data[features])
test_x = min_max_scaler.transform(test_data[features])

In [15]:
from sklearn.model_selection import train_test_split

y = train_data["price"].values

train_X, eval_X, train_Y, eval_Y = train_test_split(train_x, y, test_size = 0.2)

In [16]:
from tensorflow import keras 

model = keras.Sequential([
  keras.layers.Dense(128, activation='relu', input_shape=[len(features)]),
  keras.layers.Dense(128, activation='relu'),
  keras.layers.Dense(128, activation='relu'),
  keras.layers.Dense(1)
]
)

# adam = keras.optimizers.Adam(lr=0.01, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
model.compile(loss='mean_absolute_error', optimizer="adam")

model.fit(train_X, train_Y, batch_size=1024, epochs=200)

Epoch 1/200
118/118 [==============================] - 0s 2ms/step - loss: 5243.1660
Epoch 2/200
118/118 [==============================] - 0s 2ms/step - loss: 3912.0698
Epoch 3/200
118/118 [==============================] - 0s 2ms/step - loss: 2214.1033
Epoch 4/200
118/118 [==============================] - 0s 2ms/step - loss: 1327.4607
Epoch 5/200
118/118 [==============================] - 0s 2ms/step - loss: 1138.6050
Epoch 6/200
118/118 [==============================] - 0s 2ms/step - loss: 1055.1417
Epoch 7/200
118/118 [==============================] - 0s 2ms/step - loss: 997.0692
Epoch 8/200
118/118 [==============================] - 0s 2ms/step - loss: 952.2555
Epoch 9/200
118/118 [==============================] - 0s 2ms/step - loss: 916.7003
Epoch 10/200
118/118 [==============================] - 0s 2ms/step - loss: 886.4775
Epoch 11/200
118/118 [==============================] - 0s 2ms/step - loss: 858.9488
Epoch 12/200
118/118 [==============================] - 0s 2ms/step 

In [17]:
from sklearn.metrics import mean_absolute_error

eval_mae = mean_absolute_error(model.predict(eval_X), eval_Y)
print("evaluation dataset MAE：", eval_mae)

evaluation dataset MAE： 533.5699975220522


# 使用全量训练数据训练模型，对测试集预测并保存结果

In [18]:
model_2 = keras.Sequential([
  keras.layers.Dense(128, activation='relu', input_shape=[len(features)]),
  keras.layers.Dense(128, activation='relu'),
  keras.layers.Dense(128, activation='relu'),
  keras.layers.Dense(1)
]
)

model_2.compile(loss='mean_absolute_error', optimizer="adam")

model_2.fit(train_x, y, batch_size=1024, epochs=200)

Epoch 1/200
147/147 [==============================] - 0s 2ms/step - loss: 5072.2886
Epoch 2/200
147/147 [==============================] - 0s 2ms/step - loss: 3423.7266
Epoch 3/200
147/147 [==============================] - 0s 2ms/step - loss: 1609.8428
Epoch 4/200
147/147 [==============================] - 0s 2ms/step - loss: 1196.8563
Epoch 5/200
147/147 [==============================] - 0s 2ms/step - loss: 1087.0588
Epoch 6/200
147/147 [==============================] - 0s 2ms/step - loss: 1016.0975
Epoch 7/200
147/147 [==============================] - 0s 2ms/step - loss: 961.8803
Epoch 8/200
147/147 [==============================] - 0s 2ms/step - loss: 917.4738
Epoch 9/200
147/147 [==============================] - 0s 2ms/step - loss: 882.0532
Epoch 10/200
147/147 [==============================] - 0s 2ms/step - loss: 852.3053
Epoch 11/200
147/147 [==============================] - 0s 2ms/step - loss: 826.5662
Epoch 12/200
147/147 [==============================] - 0s 2ms/step 

In [19]:
predict_test = model_2.predict(test_x)

result = pd.DataFrame()
result['SaleID'] = test_data['SaleID']
result['price'] = predict_test
result['price'][result['price'] < 11] = 11
# result.to_csv("./drive/My Drive/Colab Notebooks/ans.csv")

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [24]:
result.to_csv("./drive/My Drive/Colab Notebooks/ans.csv")

最终得分 score:494.3832